# Questions:
## 1. What's the filter on the key words? 
### Here's the text of the first tweet, looks it's not that relevant to Boeing.
### "RT @SheliaBurk: @BillWrh1970 @bfraser747 @KellyannePolls He has good relations with Muslims though, doesn't he? How could people no\u2026 "
## 2. How to remove emoticons like :) :D ?
## 3. What is the time zone of `timestamp ms` and `created_at`?

In [27]:
import pickle
import os
from datetime import datetime
from dateutil import tz



In [2]:
path = '/Users/ggao/Documents/boeing777/boeing/'

In [ ]:
# Q1: what's the filter? why they're relevant?
# First tweet text:
# "RT @SheliaBurk: @BillWrh1970 @bfraser747 @KellyannePolls He has good relations with Muslims though, doesn't he? How could people no\u2026 "

In [41]:
file_list = os.listdir(path)
# Just to run a small sample first
file_list = [file_list[0]]

In [47]:
data = []

In [48]:
for i in range(len(file_list)):
    with open(path + file_list[i]) as input_file:
        temp_data = pickle.load(input_file)
    data += temp_data
    

In [49]:
len(data)

282

In [28]:
### Need to double check the timezone of 'timestamp_ms' and 'created_at'
from_zone = tz.gettz('UTC')
to_zone = tz.gettz('America/New_York')

# utc = datetime.utcnow()
utc = datetime.fromtimestamp(int(data[0]['timestamp_ms']) / 1000)

# Tell the datetime object that it's in UTC time zone since 
# datetime objects are 'naive' by default
utc = utc.replace(tzinfo=from_zone)

# Convert time zone
central = utc.astimezone(to_zone)

In [89]:
data[0][u'entities'].keys()

[u'user_mentions', u'symbols', u'hashtags', u'urls']

In [93]:
data[0][u'entities']['user_mentions']

[{u'id': 889148660,
  u'id_str': u'889148660',
  u'indices': [3, 14],
  u'name': u'\U0001f1fa\U0001f1f8Shelia Burk\U0001f1fa\U0001f1f8',
  u'screen_name': u'SheliaBurk'},
 {u'id': 347291458,
  u'id_str': u'347291458',
  u'indices': [16, 28],
  u'name': u'Bill (WildBill)',
  u'screen_name': u'BillWrh1970'},
 {u'id': 274891222,
  u'id_str': u'274891222',
  u'indices': [29, 40],
  u'name': u'Brian Fraser',
  u'screen_name': u'bfraser747'},
 {u'id': 471672239,
  u'id_str': u'471672239',
  u'indices': [41, 56],
  u'name': u'Kellyanne Conway',
  u'screen_name': u'KellyannePolls'}]

In [87]:
data[0].keys()

[u'contributors',
 u'truncated',
 u'text',
 u'is_quote_status',
 u'in_reply_to_status_id',
 u'id',
 u'favorite_count',
 u'source',
 u'retweeted',
 u'coordinates',
 u'timestamp_ms',
 u'entities',
 u'in_reply_to_screen_name',
 u'in_reply_to_user_id',
 u'retweet_count',
 u'id_str',
 u'favorited',
 u'retweeted_status',
 u'user',
 u'geo',
 u'in_reply_to_user_id_str',
 u'lang',
 u'created_at',
 u'filter_level',
 u'in_reply_to_status_id_str',
 u'place']

In [25]:
### Default timestamp 
datetime.fromtimestamp(int(data[1]['timestamp_ms']) / 1000)

datetime.datetime(2017, 1, 1, 0, 34, 16)

In [ ]:
# For each tweet, get the time -- truncated to day? and the tokenized tweet

In [52]:
from nltk.tokenize import word_tokenize
 
tweet = 'RT @marcobonzanini: just an example! :D http://example.com #NLP'
print(word_tokenize(tweet))


['RT', '@', 'marcobonzanini', ':', 'just', 'an', 'example', '!', ':', 'D', 'http', ':', '//example.com', '#', 'NLP']


In [129]:
import re
import nltk
from nltk.corpus import stopwords
import string
 
punctuation = list(string.punctuation)
stopwords_list = stopwords.words('english') + punctuation + ['rt', 'via']

emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    # remove emojis
    emojis_re = re.compile(
    u"(\ud83d[\ude00-\ude4f])|"  # emoticons
    u"(\ud83c[\udf00-\uffff])|"  # symbols & pictographs (1 of 2)
    u"(\ud83d[\u0000-\uddff])|"  # symbols & pictographs (2 of 2)
    u"(\ud83d[\ude80-\udeff])|"  # transport & map symbols
    u"(\ud83c[\udde0-\uddff])"  # flags (iOS)
    "+", flags=re.UNICODE)
    
    tokens = emojis_re.sub(r'', s)
    # remove hash-tag and mentions 
    tokens = re.sub(r"(?:\#)" + '|' + r'(?:@[\w_]+)', '', tokens)
    # remove URL
    tokens = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+' + '|' + r'<[^>]+>', '', tokens)
    # Q2: how to remove emoticons like :) :D
    
    tokens = tokens_re.findall(tokens)
    # Filter non stopwords, and the words starting with @
    print tokens
    tokens = [item for item in tokens if item.lower() not in stopwords_list]
    return tokens
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        # If it's emoticon then do not return lowercase
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
 
tweet = 'RT @marcobonzanini: this is just an example! :D http://example.com #NLP'
print(preprocess(tweet, lowercase=True))

['RT', ':', 'this', 'is', 'just', 'an', 'example', '!', ':D', 'NLP']
['example', ':D', 'nlp']


In [70]:
# a through f: [] represents a range
# Is two consecutive characters then use two ranges [a-f][a-f]
re.split(r'[a-fA-F]', 'ajkfhdsakFhweiuhfoi')

['', 'jk', 'h', 's', 'k', 'hw', 'iuh', 'oi']

In [98]:
# \d{1,5} : digits length 1 to 5
# +: 1 or more
# .* anything
# \w alphanumeric (either letter or number)
re.sub(r'\d{1,5}\s\w+\s\w+\.', ' ', 'adjfjadklsf475 wahsington st.fdasklfdj', )

'adjfjadklsf fdasklfdj'